In [28]:
## Simple API call to NOAA

import requests
import json
from pandas.io.json import json_normalize


url = 'https://tidesandcurrents.noaa.gov/api/datagetter?'

params = {
    'begin_date':'20130101 10:00',
    'end_date':'20130201 10:24',
    'station':'8454000',
    'product':'currents',
     #'datum':'mllw',
    'units':'metric',
    'time_zone':'gmt',
    'application':'web_services',
    'format':'json'    
}

resp = requests.get(url=url, params=params)
print(resp.json())

{'error': {'message': ' Wrong Station ID: Please submit a valid station ID '}}


In [24]:
try:
    pd_metadata = json_normalize(resp.json()['metadata'])
    pd_metadata.columns = ['Station ID', 'Latitude', 'Longitude', 'Location']
    pd_data = json_normalize(resp.json()['data'])
    
except KeyError:
    print(resp.json()['error'])
    print('Error in the api input')

Error in the api input


In [20]:
pd_metadata

,Station ID,Latitude,Longitude,Location
0,8454000,41.8071,-71.4012,Providence


In [21]:
pd_data

,f,q,s,t,v
0,"0,0,0,0",v,0.003,2013-01-01 10:00,0.072
1,"0,0,0,0",v,0.003,2013-01-01 10:06,0.095
2,"0,0,0,0",v,0.003,2013-01-01 10:12,0.115
3,"0,0,0,0",v,0.004,2013-01-01 10:18,0.138
4,"0,0,0,0",v,0.004,2013-01-01 10:24,0.167
5,"0,0,0,0",v,0.004,2013-01-01 10:30,0.193
6,"0,0,0,0",v,0.003,2013-01-01 10:36,0.217
7,"0,0,0,0",v,0.005,2013-01-01 10:42,0.244
8,"0,0,0,0",v,0.006,2013-01-01 10:48,0.277
9,"0,0,0,0",v,0.007,2013-01-01 10:54,0.316
